In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

# How does the dataset looks like ? 

In [3]:
youtube_df = pd.read_feather("youtube_big.feather")
youtube_df.head()

,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count,categories_raw,top5p_cat,top1p_cat,is_invalid,top5p,top1p
0,Music,UCsv6iJMuTZc8-zs3V2Z9B6w,0,uKGGM22X8Qs,429,4,2019-09-12,194,Music,False,False,False,False,False
1,Music,UCsv6iJMuTZc8-zs3V2Z9B6w,11,jzBaT9_h0aw,377,44,2019-06-20,2668,Music,False,False,False,False,False
2,Music,UCsv6iJMuTZc8-zs3V2Z9B6w,6,giozLk4q51A,295,66,2019-05-02,4350,Music,False,False,False,False,False
3,Music,UCsv6iJMuTZc8-zs3V2Z9B6w,0,df4qDGsSDdg,338,5,2018-03-04,138,Music,False,False,False,False,False
4,Music,UCsv6iJMuTZc8-zs3V2Z9B6w,0,MNW-OvHFibs,334,9,2018-03-04,329,Music,False,False,False,False,False


In [4]:
youtube_df.count()

categories        69974760
channel_id        69974760
dislike_count     69974760
display_id        69974760
duration          69974760
like_count        69974760
upload_date       69974760
view_count        69974760
categories_raw    69974760
top5p_cat         69974760
top1p_cat         69974760
is_invalid        69974760
top5p             69974760
top1p             69974760
dtype: int64

**So we have about 70mio of lines with no null values**

## What are the categories ? 

In [56]:
categoriesCount = youtube_df[['categories','channel_id']].groupby('categories').count()
categoriesCount.columns = ['nb_of_video_per_categories']
categoriesCount.sort_values(by=['nb_of_video_per_categories'],ascending=False)

,nb_of_video_per_categories
categories,
Entertainment,19917134
Gaming,12686630
Others,10593566
News & Politics,9026408
Music,8237064
"Education, Science & Tech",5891804
Howto & Style,3621165
,989


## What are categories_raw ? 

In [35]:
categoriesRawCount = youtube_df[['categories_raw','channel_id']].groupby('categories_raw').count()
categoriesRawCount.columns = ['count']
categoriesRawCount

,count
categories_raw,
,989
"Education, Science & Tech",5891804
Entertainment,19917134
Gaming,12686630
Howto & Style,3621165
Music,8237064
News & Politics,9026408
Others,10593566


Seems to be exactly the same than the column "categories"...

## Are the display_id really unique ? 

In [31]:
 youtube_df[['display_id']][youtube_df[['display_id']].duplicated()]

,display_id
250,XOmPr1feFHU
251,1bgkJ52aU5w
585,bf58l61iUSs
622,29pOl5Y5Ycc
754,ES_HdxP3hio
...,...
69972027,8t6GeEhIr-Q
69972250,IueeqJIkYm8
69972336,OVRkhBzmDq0
69973947,ZDY9uXuOufU


Sadly no we have originally some duplicates! See example below...

In [34]:
youtube_df[youtube_df['display_id'] == 'XOmPr1feFHU']


,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count,categories_raw,top5p_cat,top1p_cat,is_invalid,top5p,top1p
247,Howto & Style,UCsv5Rytct54UTBdSKw2Nh4Q,13,XOmPr1feFHU,262,232,2012-12-22,23484,Howto & Style,False,False,False,False,False
250,Howto & Style,UCsv5Rytct54UTBdSKw2Nh4Q,13,XOmPr1feFHU,262,232,2012-12-22,23484,Howto & Style,False,False,False,False,False


## Upload Date:

In [39]:
minDate = min(youtube_df['upload_date'])
maxDate = max(youtube_df['upload_date'])
print("Info available from {} to {} ".format(minDate, maxDate))

Info available from 2005-07-14 00:00:00 to 2019-11-20 00:00:00 


## How many different channels ? 

In [57]:
nbOfChannels = len(youtube_df['channel_id'].unique())
print("There are {} different channels distributed that way (Some channel may post videos over multiple categories): ".format(nbOfChannels))


channelPerCatDF = youtube_df[['categories','channel_id']].drop_duplicates().groupby('categories').count()
channelPerCatDF.columns = ["nb_of_different_channels"]
channelPerCatDF.sort_values(by=['nb_of_different_channels'],ascending=False)


There are 144794 different channels distributed that way (Some channel may post videos over multiple categories): 


,nb_of_different_channels
categories,
Entertainment,97286
Others,56306
Music,55860
"Education, Science & Tech",32143
Gaming,31373
Howto & Style,28869
News & Politics,11233
,357


## RECAP

Recap: 
* display_id not necessary unique => filter
* categories and categories_raw seems to be identical => drop a column
* is_invalid: True si la vidéo n'est pas en anglais (en gros à filter, mais je crois que c'est déjà le cas)
* topxp: Cette vidéo est dans le top x % global (en nombre de vue) sur une période de 6 mois. Il y a la même chose avec _y pour chaque année
* topxp_cat: la même chose mais avec un group by catégories (top x % dans la catégorie de la vidéo, vu que le nombre de vues par catégorie n'est pas distribué uniformément)

2-3 petites questions: comment ca a été calculé les tops 5,1 par catégories et sans catégories ? Pourquoi c'est nécéssairement sur une période de 6 mois de  jan - juin, juillet - déc. Est-ce possible d'avoir une vidéo qui aurait pu etre dans le top si ca avait été comtabilisé de la période avril-septembre et octobre-mars à la place ? 


## QUE FAIRE ?? 